# Infrastructure As Code

In [2]:
import pandas as pd
import boto3
import json
import psycopg2
import configparser

## STEP 0: Make sure you have an AWS secret and access key

- Create a new IAM user in your AWS account
- Give it `AdministratorAccess`, From `Attach existing policies directly` Tab
- Take note of the access key and secret 
- Create the below file `aws.cfg` in the same folder as this notebook and fill the [AWS] section.


        # Filename:  aws.cfg
        [AWS]
        KEY=[YOUR_AWS_KEY]
        SECRET=[YOUR_AWS_SECRET]
        REGION=[YOUR REGION]

        [DWH] 
        DWH_CLUSTER_TYPE=multi-node
        DWH_NUM_NODES=4
        DWH_NODE_TYPE=dc2.large

        DWH_IAM_ROLE_NAME=dwhRole
        DWH_CLUSTER_IDENTIFIER=dwhCluster
        DWH_DB=dwh
        DWH_DB_USER=dwhuser
        DWH_DB_PASSWORD=Passw0rd
        DWH_PORT=5439

        [CLUSTER]
        # HOST=dwhcluster.cauecn2rh1z6.us-east-1.redshift.amazonaws.com
        HOST=dwhcluster.cchtxpodhmfa.us-west-2.redshift.amazonaws.com
        DB_NAME=dwh
        DB_USER=dwhuser
        DB_PASSWORD=Passw0rd
        DB_PORT=5439

        [IAM_ROLE]
        ARN=[REDACTED]

        [S3]
        LOG_DATA='s3://udacity-dend/log_data'
        LOG_JSONPATH='s3://udacity-dend/log_json_path.json'
        # fails on east cluster
        SONG_DATA='s3://udacity-dend/song-data'
        # use for testing
        # (56 sec, east; 23.6 west) 
        # SONG_DATA='s3://udacity-dend/song-data/A/A/'
        # (1379.746524 sec, east; 265.13, west) 
        # SONG_DATA='s3://udacity-dend/song-data/A/'

        [PARSE]
        # Set to 1 to parse files first and identify errors; 0 to process records
        VALUE=0


## Create Redshift Cluster using the AWS Python SDK

In [ ]:
### Load DWH Params from file

In [18]:
config = configparser.ConfigParser()
config.read_file(open('aws.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')
REGION                 = config.get('AWS','REGION')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE"
                  , "DWH_NUM_NODES"
                  , "DWH_NODE_TYPE"
                  , "DWH_CLUSTER_IDENTIFIER"
                  , "DWH_DB"
                  , "DWH_DB_USER"
                  , "DWH_DB_PASSWORD"
                  , "DWH_PORT"
                  , "DWH_IAM_ROLE_NAME"]
                  , "Value": 
                    [DWH_CLUSTER_TYPE
                    , DWH_NUM_NODES
                    , DWH_NODE_TYPE
                    , DWH_CLUSTER_IDENTIFIER
                    , DWH_DB
                    , DWH_DB_USER
                    , DWH_DB_PASSWORD
                    , DWH_PORT
                    , DWH_IAM_ROLE_NAME]
             })

print(REGION)

us-west-2


## Create clients for IAM, EC2, S3 and Redshift

In [19]:
s3 = boto3.resource('s3'
                        , region_name=REGION
                        , aws_access_key_id=KEY
                        , aws_secret_access_key=SECRET
                   )

ec2 = boto3.resource('ec2'
                        , region_name=REGION
                        , aws_access_key_id=KEY
                        , aws_secret_access_key=SECRET
                    )

iam = boto3.client('iam'
                        ,aws_access_key_id=KEY
                        , aws_secret_access_key=SECRET
                        , region_name=REGION
                  )

redshift = boto3.client('redshift'
                        , region_name=REGION
                        , aws_access_key_id=KEY
                        , aws_secret_access_key=SECRET
                       )

In [16]:
### Verify S3 connection- check out sample data sources on S3

In [34]:
sampleDbBucket =  s3.Bucket("awssampledbuswest2")
for obj in sampleDbBucket.objects.filter(Prefix="ssbgz"):
    print(obj)

s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/customer0002_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/dwdate.tbl.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0000_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0001_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0002_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0003_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0004_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0005_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0006_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0007_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='s

In [37]:
input_path = 'amazon-reviews-pds'
sampleDbBucket =  s3.Bucket(input_path)
for obj in sampleDbBucket.objects.filter(Prefix="parquet"):
    print(obj)

 key='parquet/product_category=PC/part-00005-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet')
s3.ObjectSummary(bucket_name='amazon-reviews-pds', key='parquet/product_category=PC/part-00006-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet')
s3.ObjectSummary(bucket_name='amazon-reviews-pds', key='parquet/product_category=PC/part-00007-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet')
s3.ObjectSummary(bucket_name='amazon-reviews-pds', key='parquet/product_category=PC/part-00008-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet')
s3.ObjectSummary(bucket_name='amazon-reviews-pds', key='parquet/product_category=PC/part-00009-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet')
s3.ObjectSummary(bucket_name='amazon-reviews-pds', key='parquet/product_category=Personal_Care_Appliances/part-00000-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet')
s3.ObjectSummary(bucket_name='amazon-reviews-pds', key='parquet/product_category=Personal_Care_Appliance

Read the Data WareHouse configuration details to create a Redshift Cluster

Create clients for EC2, S3, IAM, and RedShift

Create a local Spark session